# **Kegiatan Modul 6**

Content Based Filtering : Anime

Tulislan code sesuai instruksi pada sel yang telah disediakan

## Downlaod dan extract dataset dari situs [kaggle.com](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020)

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

100% 658M/661M [00:22<00:00, 28.2MB/s]
100% 661M/661M [00:22<00:00, 30.2MB/s]


In [ ]:
!unzip anime-recommendation-database-2020.zip

## Import library dan load dataset

In [ ]:
import pandas as pd

In [ ]:
# Untuk tugas ini cukup gunakan data "anime_with_synopsis.csv"
# Tampilkan 5 data teratas
df = pd.read_csv("anime_with_synopsis.csv")
df.head(5)

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


## Tangani data yang kosong (missing value)

In [ ]:
########################
# Preprocessing
########################
df.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64

In [ ]:
df.dropna(subset=['sypnopsis'],inplace=True)
df

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


Hasil penanganan missing value

In [ ]:
df.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    0
dtype: int64

## Selanjutnya lakukan feature extraction pada fitur "Genres" menggunakan TF-IDF

In [ ]:
########################
# Creating TF-IDF Matrix
########################
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Genres'])

tfidf_matrix.shape

(16206, 46)

## Menghitung cosine similarity dari TF-IDF diatas

In [ ]:
########################
# Creating Cosine Similarity Matrix
########################
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim.shape

(16206, 16206)

## Membuat Content Based Recomender

In [ ]:
########################
# Creating Content Based Recommender
########################

df['Name'] = df['Name'].str.lower()
df['Genres'] = df['Genres'].fillna('')

def get_recommendations(search_title):
    search_title = search_title.lower()
    anime_index = df[df['Name'] == search_title].index[0]
    
    similarity_scores = list(enumerate(cosine_sim[anime_index]))
    sorted_anime = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    top_anime_indices = [anime[0] for anime in sorted_anime if anime[0] != anime_index]
    top_anime_titles = df.iloc[top_anime_indices][['Name', 'Genres']]
    
    return top_anime_titles

## Mencoba Content Based Recomender

Mempilkan 5 Judul Anime dengan yang memiliki nilai kemiripan tertinggi berdasarkan genre dari kata kunci judul anime yang dimasukkan

In [ ]:
# hasil rekomendasi anime dengan keyword "Paprika"
anime_title = "Paprika"

recommended_anime = get_recommendations(anime_title)
recommended_anime.head(5)

,Name,Genres
824,higurashi no naku koro ni,"Mystery, Dementia, Horror, Psychological, Supe..."
15171,higurashi no naku koro ni gou,"Mystery, Dementia, Horror, Psychological, Supe..."
16211,higurashi no naku koro ni sotsu,"Mystery, Dementia, Horror, Psychological, Supe..."
13265,yakusoku no neverland,"Sci-Fi, Mystery, Horror, Psychological, Thrill..."
6598,higurashi no naku koro ni kaku: outbreak,"Mystery, Horror, Psychological, Thriller"


In [ ]:
# hasil rekomendasi anime dengan keyword "Monster"
anime_title = "Monster"

recommended_anime = get_recommendations(anime_title)
recommended_anime.head(5)

,Name,Genres
6598,higurashi no naku koro ni kaku: outbreak,"Mystery, Horror, Psychological, Thriller"
2495,death note: rewrite,"Mystery, Police, Psychological, Supernatural, ..."
10449,b: the beginning,"Action, Mystery, Police, Psychological, Supern..."
13404,b: the beginning succession,"Action, Mystery, Police, Psychological, Supern..."
289,mousou dairinin,"Mystery, Dementia, Police, Psychological, Supe..."


In [ ]:
# hasil rekomendasi anime dengan keyword "Naruto"
anime_title = "Naruto"

recommended_anime = get_recommendations(anime_title)
recommended_anime.head(5)

,Name,Genres
1508,naruto: shippuuden,"Action, Adventure, Comedy, Super Power, Martia..."
11640,boruto: jump festa 2016 special,"Action, Adventure, Comedy, Super Power, Martia..."
206,rekka no honoo,"Action, Adventure, Martial Arts, Shounen, Supe..."
5518,naruto: honoo no chuunin shiken! naruto vs. ko...,"Action, Adventure, Martial Arts, Shounen, Supe..."
6158,naruto: shippuuden movie 6 - road to ninja,"Action, Adventure, Super Power, Martial Arts, ..."
